# AI Interview System - Organized Version

This notebook implements an AI-powered interview system using LangGraph, Google Gemini, and various speech technologies.

## Features
- 🤖 AI Interviewer with customizable personality
- 📚 Knowledge base integration for position-specific questions
- 📄 Resume analysis and project-based questioning
- 🎤 Speech-to-Text (STT) for voice input
- 🔊 Text-to-Speech (TTS) for voice output
- 🔄 Real-time conversation flow with LangGraph

## Table of Contents
1. [Setup and Dependencies](#setup)
2. [Core Components](#core-components)
3. [Knowledge Base Setup](#knowledge-base)
4. [Interview Agent](#interview-agent)
5. [Speech Technologies](#speech)
6. [Integration and Testing](#integration)
7. [Usage Examples](#usage)

## 1. Setup and Dependencies <a name="setup"></a>

In [ ]:
# Install required packages
!pip install langchain_google_genai dotenv langgraph langchain_community langchain_chroma pypdf
!pip install elevenlabs assemblyai sounddevice soundfile pyaudio websocket-client scipy speech_recognition

In [ ]:
# Core imports
from langgraph.graph import StateGraph, START, END
from typing import Dict, TypedDict, List, Union, Annotated, Sequence
from IPython.display import display, Markdown, Image, Audio
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, AnyMessage, BaseMessage, ToolMessage
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langgraph.graph.message import add_messages
from langchain_chroma import Chroma
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain.prompts import PromptTemplate
from langchain.tools.retriever import create_retriever_tool
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

print("✅ All imports loaded successfully")

In [ ]:
# Initialize LLM and embeddings
def initialize_models():
    """Initialize the language model and embeddings."""
    try:
        llm = init_chat_model("google_genai:gemini-2.5-flash-lite-preview-06-17")
        embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
        
        # Test the models
        test_response = llm.invoke(input=[HumanMessage(content="Hello")])
        test_embedding = embeddings.embed_query("Hello")
        
        print(f"✅ LLM initialized successfully")
        print(f"✅ Embeddings initialized successfully (dimension: {len(test_embedding)})")
        
        return llm, embeddings
    except Exception as e:
        print(f"❌ Error initializing models: {e}")
        raise

llm, embeddings = initialize_models()

## 2. Core Components <a name="core-components"></a>

In [ ]:
# Define the agent state structure
class AgentState(TypedDict):
    """
    State management for the recruiter agent.
    
    Attributes:
        mode: Interview style (friendly, formal, technical)
        num_of_q: Number of main questions to ask
        num_of_follow_up: Number of follow-up questions allowed
        position: Job position being interviewed for
        company_name: Company conducting the interview
        messages: Conversation history
    """
    mode: str
    num_of_q: int
    num_of_follow_up: int
    position: str
    company_name: str
    messages: Annotated[list, add_messages]

## 3. Knowledge Base Setup <a name="knowledge-base"></a>

In [ ]:
def setup_knowledge_base():
    """
    Set up the knowledge base with interview questions and resume data.
    
    Returns:
        tuple: (questions_retriever, resume_retriever)
    """
    # File paths
    pdf_path = "../utils/LLM Interview Questions.pdf"
    resume_path = "../utils/Mohamed-Mowina-AI-Resume.pdf"
    
    # Validate file existence
    for path in [pdf_path, resume_path]:
        if not os.path.exists(path):
            raise FileNotFoundError(f"Required file not found: {path}")
    
    try:
        # Load documents
        pdf_loader = PyPDFLoader(pdf_path)
        resume_loader = PyPDFLoader(resume_path)
        
        pages = pdf_loader.load()
        resume = resume_loader.load()
        
        print(f"✅ Loaded {len(pages)} pages from interview questions")
        print(f"✅ Loaded {len(resume)} pages from resume")
        
        # Split documents into chunks
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        
        pages_split = text_splitter.split_documents(pages)
        resume_split = text_splitter.split_documents(resume)
        
        print(f"✅ Split into {len(pages_split)} question chunks")
        print(f"✅ Split into {len(resume_split)} resume chunks")
        
        # Create vector stores
        questions_vectorstore = Chroma.from_documents(
            documents=pages_split, 
            embedding=embeddings, 
            collection_name="interview_questions", 
            persist_directory="./data/interview_questions"
        )
        
        resume_vectorstore = Chroma.from_documents(
            documents=resume_split, 
            embedding=embeddings, 
            collection_name="resume", 
            persist_directory="./data/resume"
        )
        
        # Create retrievers
        questions_retriever = questions_vectorstore.as_retriever()
        resume_retriever = resume_vectorstore.as_retriever()
        
        print("✅ Knowledge base setup complete")
        
        return questions_retriever, resume_retriever
        
    except Exception as e:
        print(f"❌ Error setting up knowledge base: {e}")
        raise

# Setup knowledge base
questions_retriever, resume_retriever = setup_knowledge_base()

In [ ]:
# Create retriever tools
def create_tools():
    """Create the retriever tools for the agent."""
    questions_tool = create_retriever_tool(
        questions_retriever,
        "retrieve_questions",
        "Search and return questions related to the position from the knowledge base."
    )
    
    resume_tool = create_retriever_tool(
        resume_retriever,
        "retrieve_resume",
        "Search resume and return related projects and experience relevant to the position."
    )
    
    return [questions_tool, resume_tool]

tools = create_tools()
print(f"✅ Created {len(tools)} tools")

## 4. Interview Agent <a name="interview-agent"></a>

In [ ]:
# Define the interviewer prompt template
interviewer_prompt = PromptTemplate(
    input_variables=["mode", "company_name", "position", "number_of_questions", "number_of_followup"],
    template="""
You are a {mode} AI interviewer for {company_name}, conducting an interview for a {position} position.

Your goal is to assess the candidate's technical skills, problem-solving abilities, communication skills, and experience relevant to the position.

You have access to two tools:
1. `retrieve_questions`: Search for position-specific interview questions
2. `retrieve_resume`: Search the candidate's resume for relevant projects and experience

Interview Structure:
- Start with a friendly introduction
  - Ask the candidate to introduce themselves
  - Ask about a specific project from their resume
- Ask {number_of_questions} main questions from the knowledge base
- Ask up to {number_of_followup} follow-up questions if answers are vague
- End with "Thank you, that's it for today."

Guidelines:
- Maintain a {mode} tone throughout
- Number your questions clearly (Question 1, Question 2, etc.)
- If asked irrelevant questions, respond with "Sorry, this is out of scope."
- Print "tool used: `tool_name`" when using tools

Begin the interview now.
"""
)

In [ ]:
def recruiter_agent(state: AgentState) -> AgentState:
    """
    The main recruiter agent function that processes the conversation.
    
    Args:
        state: Current agent state
        
    Returns:
        Updated agent state
    """
    # Create system prompt
    sys_prompt = SystemMessage(content=interviewer_prompt.format(
        mode=state['mode'],
        company_name=state['company_name'],
        position=state['position'],
        number_of_questions=state['num_of_q'],
        number_of_followup=state['num_of_follow_up']
    ))
    
    # Prepare messages for LLM
    all_messages = [sys_prompt] + state["messages"]
    
    # Invoke LLM with tools
    response = llm.bind_tools(tools).invoke(all_messages)
    
    return {"messages": response}

def should_continue(state: AgentState) -> str:
    """
    Determine the next step in the conversation flow.
    
    Args:
        state: Current agent state
        
    Returns:
        Next action: 'invoke_tools', 'continue_convo', or 'end'
    """
    last_msg = state["messages"][-1]
    
    # Check if agent wants to use tools
    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        return "invoke_tools"
    
    # Check if interview is finished
    if "Thank you, that's it for today." in last_msg.content:
        return "end"
    
    # Continue conversation
    return "continue_convo"

In [ ]:
# Build the LangGraph workflow
def build_workflow():
    """Build and compile the LangGraph workflow."""
    # Create the graph
    workflow = StateGraph(AgentState)
    
    # Add nodes
    workflow.add_node("recruiter", recruiter_agent)
    
    # Add tool node
    tool_node = ToolNode(tools)
    workflow.add_node("tools", tool_node)
    
    # Set entry point
    workflow.set_entry_point("recruiter")
    
    # Add conditional edges
    workflow.add_conditional_edges(
        "recruiter",
        should_continue,
        {
            "invoke_tools": "tools",
            "continue_convo": "recruiter",
            "end": END
        }
    )
    
    # Add edge from tools back to recruiter
    workflow.add_edge("tools", "recruiter")
    
    # Compile the graph
    app = workflow.compile()
    
    return app

# Build the workflow
app = build_workflow()
print("✅ LangGraph workflow built successfully")

# Display the workflow diagram
try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"⚠️ Could not display workflow diagram: {e}")

## 5. Speech Technologies <a name="speech"></a>

In [ ]:
# Text-to-Speech (TTS) with ElevenLabs
def setup_tts():
    """Setup ElevenLabs TTS client."""
    try:
        from elevenlabs import ElevenLabs
        
        client = ElevenLabs(api_key=os.getenv("ELEVENLABS_API_KEY"))
        
        # Test TTS
        test_audio = client.text_to_speech.convert(
            text="TTS system initialized successfully.",
            voice_id="JBFqnCBsd6RMkjVDRZzb",
            model_id="eleven_multilingual_v2",
            output_format="mp3_44100_128"
        )
        
        print("✅ ElevenLabs TTS initialized successfully")
        return client
        
    except Exception as e:
        print(f"❌ Error setting up TTS: {e}")
        return None

tts_client = setup_tts()

In [ ]:
# Speech-to-Text (STT) with AssemblyAI
def setup_stt():
    """Setup AssemblyAI STT client."""
    try:
        import assemblyai as aai
        
        aai.settings.api_key = os.getenv("ASSEMBLYAI_API_KEY")
        transcriber = aai.Transcriber()
        
        print("✅ AssemblyAI STT initialized successfully")
        return transcriber
        
    except Exception as e:
        print(f"❌ Error setting up STT: {e}")
        return None

stt_transcriber = setup_stt()

In [ ]:
# Audio recording and transcription function
def record_and_transcribe(duration=5, sample_rate=16000):
    """
    Record audio and transcribe using AssemblyAI.
    
    Args:
        duration: Recording duration in seconds
        sample_rate: Audio sample rate
        
    Returns:
        Transcribed text or empty string if failed
    """
    if not stt_transcriber:
        print("❌ STT not available")
        return ""
    
    try:
        import sounddevice as sd
        import tempfile
        from scipy.io.wavfile import write as write_wav
        
        print(f"🎙️ Recording for {duration} seconds...")
        
        # Record audio
        audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='int16')
        sd.wait()
        print("✅ Recording complete")
        
        # Save to temporary file
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
            write_wav(tmp.name, sample_rate, audio)
            audio_path = tmp.name
        
        # Transcribe
        transcript = stt_transcriber.transcribe(audio_path)
        
        # Clean up
        os.unlink(audio_path)
        
        if transcript.status == aai.TranscriptStatus.completed:
            print(f"🗣️ Transcription: {transcript.text}")
            return transcript.text
        else:
            print(f"❌ Transcription failed: {transcript.error}")
            return ""
            
    except Exception as e:
        print(f"❌ Error in recording/transcription: {e}")
        return ""

# Test STT
print("Testing STT - please speak when prompted...")
test_transcript = record_and_transcribe()
if test_transcript:
    print(f"✅ STT test successful: '{test_transcript}'")
else:
    print("❌ STT test failed")

## 6. Integration and Testing <a name="integration"></a>

In [ ]:
# Integrated chat loop with voice capabilities
def voice_chat_loop(initial_state: AgentState, use_voice=True):
    """
    Run a conversational loop with optional voice input/output.
    
    Args:
        initial_state: Initial agent state
        use_voice: Whether to use voice input/output
        
    Returns:
        Final conversation state
    """
    current_state = initial_state
    
    print("\n🎯 Starting AI Interview System")
    print(f"📝 Mode: {current_state['mode']}")
    print(f"🏢 Company: {current_state['company_name']}")
    print(f"💼 Position: {current_state['position']}")
    print(f"🎤 Voice enabled: {use_voice}")
    print("\n" + "="*50)
    
    try:
        while True:
            # Get user input
            if use_voice:
                print("\n🎤 Speak now (or type 'quit' to exit)...")
                user_input = record_and_transcribe()
                if not user_input:
                    print("⚠️ No speech detected, please try again")
                    continue
            else:
                user_input = input("\nYou: ")
            
            # Check for exit command
            if user_input.lower() in ['quit', 'exit', 'stop']:
                print("\n👋 Ending interview session")
                break
            
            print(f"You: {user_input}")
            
            # Add user message to state
            current_state["messages"].append(HumanMessage(content=user_input))
            
            # Run the workflow
            result = app.invoke(current_state)
            current_state = result
            
            # Get AI response
            ai_message = current_state["messages"][-1]
            
            # Display AI response
            print("\n🤖 AI Recruiter:")
            ai_message.pretty_print()
            
            # Generate and play audio if TTS is available
            if use_voice and tts_client and isinstance(ai_message, AIMessage):
                try:
                    audio_stream = tts_client.text_to_speech.convert(
                        text=ai_message.content,
                        voice_id="JBFqnCBsd6RMkjVDRZzb",
                        model_id="eleven_multilingual_v2",
                        output_format="mp3_44100_128"
                    )
                    
                    # Collect audio bytes and play
                    audio_bytes = b"".join(list(audio_stream))
                    display(Audio(audio_bytes, autoplay=True))
                    
                except Exception as e:
                    print(f"⚠️ TTS error: {e}")
            
            print("\n" + "="*50)
            
    except KeyboardInterrupt:
        print("\n❌ Interview interrupted by user")
    except Exception as e:
        print(f"\n❌ Error in chat loop: {e}")
    
    return current_state

In [ ]:
# Test the system with a simple configuration
def test_system():
    """Test the interview system with a basic configuration."""
    test_state = AgentState(
        mode="friendly",
        num_of_q=2,
        num_of_follow_up=1,
        position="AI Developer",
        company_name="TechCorp",
        messages=[HumanMessage(content="Hello, I'm ready for the interview.")]
    )
    
    print("🧪 Testing interview system...")
    
    # Test with text input first
    final_state = voice_chat_loop(test_state, use_voice=False)
    
    print("\n📊 Interview Summary:")
    print(f"Total messages: {len(final_state['messages'])}")
    
    return final_state

# Uncomment to run test
# test_result = test_system()

## 7. Usage Examples <a name="usage"></a>

In [ ]:
# Example 1: Text-based interview
def run_text_interview():
    """Run a text-based interview session."""
    initial_state = AgentState(
        mode="professional",
        num_of_q=3,
        num_of_follow_up=2,
        position="Machine Learning Engineer",
        company_name="AI Innovations Inc.",
        messages=[]
    )
    
    return voice_chat_loop(initial_state, use_voice=False)

# Example 2: Voice-based interview
def run_voice_interview():
    """Run a voice-based interview session."""
    initial_state = AgentState(
        mode="friendly",
        num_of_q=2,
        num_of_follow_up=1,
        position="Data Scientist",
        company_name="DataTech Solutions",
        messages=[]
    )
    
    return voice_chat_loop(initial_state, use_voice=True)

# Example 3: Custom interview configuration
def run_custom_interview(mode="friendly", position="AI Developer", company="OpenAI", questions=5, followups=2):
    """Run a custom interview with specified parameters."""
    initial_state = AgentState(
        mode=mode,
        num_of_q=questions,
        num_of_follow_up=followups,
        position=position,
        company_name=company,
        messages=[]
    )
    
    return voice_chat_loop(initial_state, use_voice=True)

print("✅ Usage examples defined")
print("\nAvailable functions:")
print("- run_text_interview(): Text-based interview")
print("- run_voice_interview(): Voice-based interview")
print("- run_custom_interview(): Custom configuration")

## Quick Start Guide

To start an interview session, run one of the following:

```python
# Text-based interview
result = run_text_interview()

# Voice-based interview
result = run_voice_interview()

# Custom interview
result = run_custom_interview(
    mode="technical",
    position="Senior AI Engineer",
    company="Google",
    questions=4,
    followups=3
)
```

## Configuration

Make sure you have the following environment variables set in your `.env` file:

- `GOOGLE_API_KEY`: For Gemini LLM
- `ELEVENLABS_API_KEY`: For text-to-speech
- `ASSEMBLYAI_API_KEY`: For speech-to-text

## Troubleshooting

1. **STT not working**: Check your microphone permissions and AssemblyAI API key
2. **TTS not working**: Verify your ElevenLabs API key and internet connection
3. **LLM errors**: Ensure your Google API key is valid and has sufficient quota
4. **File not found**: Make sure the PDF files are in the correct location (`../utils/`)

## Next Steps

This system can be extended with:
- Real-time emotion detection
- Interview scoring and feedback
- Multi-language support
- Integration with HR systems
- Advanced analytics and reporting